In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import websockets
import json
import datetime

async def send_audio():
    uri = "wss://dramatic-curve-urban-markets.trycloudflare.com/ws"

    # ping_interval=None 으로 타임아웃 해제 + max_size 확장
    async with websockets.connect(
        uri,
        ping_interval=None,
        max_size=10 * 1024 * 1024  # 10MB까지 허용
    ) as websocket:
        # 1) JSON 메타데이터
        meta = {
            "user_id": "test_user",
            "sequence": 1,
            "timestamp": datetime.datetime.utcnow().isoformat()
        }
        await websocket.send(json.dumps(meta))
        print("✅ 메타데이터 전송 완료")

        # 2) 바이너리(테스트용 WAV 파일)
        with open("test.wav", "rb") as f:
            audio_data = f.read()
        await websocket.send(audio_data)
        print("✅ 오디오 데이터 전송 완료")

        # 3) JSON 응답 수신
        response_meta = await websocket.recv()
        response_json = json.loads(response_meta)
        print(f"✅ 응답 메타데이터: {response_json}")

        # 4) 음성 chunk 수신
        pcm_data = await websocket.recv()
        print(f"✅ PCM 데이터 수신: {len(pcm_data)} bytes")

        # 5) 저장 (테스트용)
        with open("received_from_server.wav", "wb") as f:
            f.write(pcm_data)

        print("🎧 저장 완료. 재생해보세요!")

# asyncio 이벤트루프 실행
await send_audio()
